# Data Ingestion

## Copy CSV files to S3

In [ ]:
# Assume all the pre-requisites were set up 
%store -r setup_instance_check_passed
%store -r setup_dependencies_passed
%store -r setup_s3_bucket_passed
%store -r setup_iam_roles_passed


In [1]:
# Load Libraries
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [16]:
# Public Data
!aws s3 ls s3://ads508team5/

# Copy Datasets from :
# Source of data
s3_public_path_tweeter = "s3://ads508team5/tweeter"
s3_public_path_nyt = "s3://ads508team5/nyt"
s3_public_path_cities = "s3://ads508team5/cities"

# Destination Data:
s3_private_path_tweeter = "s3://{}/ADS508_project/tweeter".format(bucket)
s3_private_path_nyt = "s3://{}/ADS508_project/nyt".format(bucket)
s3_private_path_cities = "s3://{}/ADS508_project/cities".format(bucket)


# Copy datasets
!aws s3 cp --recursive $s3_public_path_tweeter/ $s3_private_path_tweeter/ --exclude "*" --include "hashtag_donaldtrump.csv"
!aws s3 cp --recursive $s3_public_path_tweeter/ $s3_private_path_tweeter/ --exclude "*" --include "hashtag_joebiden.csv"
!aws s3 cp --recursive $s3_public_path_nyt/ $s3_private_path_nyt/ --exclude "*" --include "nyt-comments-2020.csv"
!aws s3 cp --recursive $s3_public_path_cities/ $s3_private_path_cities/ --exclude "*" --include "uscities.csv"

                           PRE cities/
                           PRE nyt/
                           PRE tweeter/
copy: s3://ads508team5/tweeter/hashtag_donaldtrump.csv to s3://sagemaker-us-west-2-471112815505/ADS508_project/tweeter/hashtag_donaldtrump.csv
copy: s3://ads508team5/tweeter/hashtag_joebiden.csv to s3://sagemaker-us-west-2-471112815505/ADS508_project/tweeter/hashtag_joebiden.csv
copy: s3://ads508team5/nyt/nyt-comments-2020.csv to s3://sagemaker-us-west-2-471112815505/ADS508_project/nyt/nyt-comments-2020.csv
copy: s3://ads508team5/cities/uscities.csv to s3://sagemaker-us-west-2-471112815505/ADS508_project/cities/uscities.csv


## Create Database, Tables and Parquets
### Database

In [2]:
# Setup and check pre-requisites to create Database
ingest_create_athena_db_passed = False

!pip install --disable-pip-version-check -q PyAthena==2.1.0
from pyathena import connect

In [3]:

# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# Create Connection
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [4]:
# Create Database
database_name = "dbpoliticpulsecomment"

statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)

pd.read_sql(statement, conn)

# Verify DB successfully created
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)


/tmp/ipykernel_30253/776241972.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)
/tmp/ipykernel_30253/776241972.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,dbpoliticpulsecomment
1,default
2,dsoaws
3,sagemaker_featurestore


In [ ]:
# End of Create Database
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

### Tables

In [ ]:
# Create Tweeter Tables
table_name_csv = "tweeter"
s3_private_path_tweeter = "s3://{}/ADS508_project/tweeter".format(bucket)

statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
  created_at TIMESTAMP,
  tweet_id FLOAT,
  tweet VARCHAR(250),
  likes INT,
  retweet_count INT,
  source VARCHAR(45),
  user_id INT,
  user_name VARCHAR(250),
  user_screen_name VARCHAR(45),
  user_description VARCHAR(250),
  user_join_date TIMESTAMP,
  user_followers_count INT,
  user_location VARCHAR(45),
  lat FLOAT,
  long FLOAT,
  city VARCHAR(45),
  country VARCHAR(45),
  continent VARCHAR(45),
  state VARCHAR(45),
  state_code VARCHAR(45),
  collected_at VARCHAR(45)
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_tweeter
)

pd.read_sql(statement, conn)


In [ ]:
# testing hashtag_donaldtrump.csv tweet
tweet = "You get a tie! And you get a tie! #Trump ‘s rally #Iowa https://t.co/jJalUUmh5D"
# testing hashtag_joebiden.csv tweet
tweet = "@chrislongview Watching and setting dvr. Let’s give him bonus ratings!! #JoeBiden"


statement = """SELECT * FROM {}.{}
    WHERE tweet = '{}' LIMIT 100""".format(
    database_name, table_name_csv, tweet
)

df = pd.read_sql(statement, conn)
df.head(5)


In [ ]:
# Create NYT comment Table
table_name_csv = "nyt_comment"
s3_private_path_nyt = "s3://{}/ADS508_project/nyt".format(bucket)


statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
  commentID INT,
  status VARCHAR(20),
  commentSequence INT,
  userID INT,
  userDisplayName VARCHAR(45),
  userLocation VARCHAR(45),
  userTitle VARCHAR(10),
  commentBody VARCHAR(500),
  createDate TIMESTAMP,
  updateDate TIMESTAMP,
  approveDate TIMESTAMP,
  recommendation INT,
  replyCount INT,
  editorsSelection VARCHAR(20),
  parentID INT,
  parentUserDisplayName VARCHAR(45),
  depth INT,
  commentType VARCHAR(20),
  trusted VARCHAR(20),
  recommendedFlag VARCHAR(20),
  permID INT,
  isAnonymous VARCHAR(20),
  articleID VARCHAR(150)
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_nyt
)

pd.read_sql(statement, conn)


In [ ]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

In [ ]:
commentBody = "@Philip Brown Agree 110%."

statement = """SELECT * FROM {}.{}
    WHERE commentBody = '{}' LIMIT 100""".format(
    database_name, table_name_csv, commentBody
)

df = pd.read_sql(statement, conn)
df.head(5)


In [21]:
# Create uscities table
table_name = "uscities"
s3_private_path_cities = "s3://{}/ADS508_project/cities".format(bucket)

statement = """
CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
    city VARCHAR(45),
    state_id VARCHAR(2),
    state_name VARCHAR(30)
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')
""".format(
    database_name, table_name, s3_private_path_cities
)

pd.read_sql(statement, conn)


/tmp/ipykernel_97/2896368081.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


### Parquets
#### Feature Selection and Creation
Prior creating a parquet table, tweeter and NYT comment tables are going to be combined under Comment Table in Athena where a feature creation is going to be performed. When the table is combined, we selected some features that would be benefit to our goal.

Selected features which could help on train the model are comment_body, user_location, candidatepoll

In [ ]:
table_comment = "comment"
table_tweeter = "tweeter"
table_nyt_comment = "nyt_comment"

# SQL Statement combine tweeter and NYT comment in the preparation to create parquet
statement = """
CREATE TABLE IF NOT EXISTS {}.{} AS (
SELECT DISTINCT
    tweet_id AS comment_id,
    tweet AS comment_body,
    user_id AS user_id,
    user_name AS user_name,
    user_location AS user_location,
    u.state_id AS user_state,
    created_at AS create_date,
    retweet_count AS reply_retweet,
    likes AS recommendation_like,
    'tweet' as source,
    (LENGTH(tweet) - LENGTH(REPLACE(LOWER(tweet), 'trump', ''))) / LENGTH('trump') AS trump_count,
    (LENGTH(tweet) - LENGTH(REPLACE(LOWER(tweet), 'biden', ''))) / LENGTH('biden') AS biden_count,
    CASE 
        WHEN 
            (LENGTH(tweet) - LENGTH(REPLACE(LOWER(tweet), 'biden', ''))) / LENGTH('biden') > 
            (LENGTH(tweet) - LENGTH(REPLACE(LOWER(tweet), 'trump', ''))) / LENGTH('trump') 
        THEN 'Biden'
        WHEN              
            (LENGTH(tweet) - LENGTH(REPLACE(LOWER(tweet), 'biden', ''))) / LENGTH('biden') < 
            (LENGTH(tweet) - LENGTH(REPLACE(LOWER(tweet), 'trump', ''))) / LENGTH('trump') 
        THEN 'Trump'
        ELSE NULL
    END AS candidatepoll
FROM
    {}.{}
LEFT JOIN
    dbpoliticpulsecomment.uscities u ON lower(user_location) = lower(u.city) OR 
    lower(user_location) = lower(u.state_name) OR 
    lower(user_location) = lower(u.state_id)
WHERE
    LENGTH(tweet) > 3
UNION
SELECT DISTINCT
    commentid AS comment_id,
    commentbody AS comment_body,
    userID AS user_id,
    userDisplayName AS user_name,
    userLocation AS user_location,
    u.state_id as user_state,
    createDate AS create_date,
    replyCount AS reply_retweet,
    recommendation AS recommendation_like,
    'nyt_comment' as source,
    (LENGTH(commentbody) - LENGTH(REPLACE(LOWER(commentbody), 'trump', ''))) / LENGTH('trump') AS trump_count,
    (LENGTH(commentbody) - LENGTH(REPLACE(LOWER(commentbody), 'biden', ''))) / LENGTH('biden') AS biden_count,
    CASE
        WHEN
            (LENGTH(commentbody) - LENGTH(REPLACE(LOWER(commentbody), 'biden', ''))) / LENGTH('biden') > 
            (LENGTH(commentbody) - LENGTH(REPLACE(LOWER(commentbody), 'trump', ''))) / LENGTH('trump') 
        THEN 'Biden'
        WHEN
            (LENGTH(commentbody) - LENGTH(REPLACE(LOWER(commentbody), 'biden', ''))) / LENGTH('biden') < 
            (LENGTH(commentbody) - LENGTH(REPLACE(LOWER(commentbody), 'trump', ''))) / LENGTH('trump') 
        THEN 'Trump'
        ELSE NULL
    END AS candidatepoll
FROM 
    {}.{}
LEFT JOIN
    dbpoliticpulsecomment.uscities u ON lower(userLocation) = lower(u.city) OR 
    lower(userLocation) = lower(u.state_name) OR 
    lower(userLocation) = lower(u.state_id)
WHERE LENGTH(commentbody) > 3
)""".format(database_name, table_comment,database_name, table_tweeter,database_name, table_nyt_comment)

pd.read_sql(statement, conn)

In [ ]:
commentbody = "Joe Biden"
source = "nyt_comment"

statement = """SELECT * FROM {}.{}
    WHERE comment_body like '{}%' and source = '{}' LIMIT 100""".format(
    database_name, table_comment, commentbody, source
)

df = pd.read_sql(statement, conn)
df.head(5)


In [29]:
# Setup to create Parquet
ingest_create_athena_table_parquet_passed = False

# Set S3 path to Parquet data
s3_path_parquet = "s3://{}/ADS508_project/parquet".format(bucket)

table_parquet = "comment_parquet"

In [ ]:
# SQL statement to execute (remove comment_id, user_name,)
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['candidatepoll']) AS
SELECT DISTINCT user_location,
         user_state,
         comment_body,
         source,
         candidatepoll
FROM {}.{}
where candidatepoll is not null AND Length(comment_body)>1 AND length(user_location)>1""".format(
    database_name, table_parquet, s3_path_parquet, database_name, table_comment
)
pd.read_sql(statement, conn)

In [30]:
#load the Parquet partitions
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, table_parquet)

df = pd.read_sql(statement, conn)
df.head(5)

/tmp/ipykernel_4319/2260227216.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


""


In [31]:
statement = "SHOW PARTITIONS {}.{}".format(database_name, table_parquet)
df_partitions = pd.read_sql(statement, conn)
df_partitions.head(5)

/tmp/ipykernel_4319/680368856.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_partitions = pd.read_sql(statement, conn)


,partition
0,candidatepoll=Biden
1,candidatepoll=Trump


In [32]:
statement = "SHOW TABLES in {}".format(database_name)
df_partitions = pd.read_sql(statement, conn)
df_partitions.head(5)

/tmp/ipykernel_4319/2954009692.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_partitions = pd.read_sql(statement, conn)


,tab_name
0,comment
1,comment_parquet
2,nyt_comment
3,tweeter
4,uscities


In [ ]:
candidatepoll = "Trump"

statement = """SELECT * FROM {}.{}
    Where candidatepoll = '{}' LIMIT 10""".format(
    database_name, table_parquet, candidatepoll
)
df_parquet = pd.read_sql(statement, conn)
df_parquet.head(5)

## Query Data

In [5]:
# Setup and check pre-requisites to create Database

!pip install --disable-pip-version-check -q awswrangler
import awswrangler as wr

In [6]:
t_filter = lambda x: x["candidatepoll"] == "Trump"
b_filter = lambda x: x["candidatepoll"] == "Biden"

In [7]:
path =  "s3://{}/ADS508_project/parquet".format(bucket)

df_parquet_results = wr.s3.read_parquet(
    path, columns=["comment_body", "source", "user_location", "candidatepoll"], partition_filter=t_filter, dataset=True
)

#"user_state", 


In [8]:
df_parquet_results_trump = df_parquet_results.drop_duplicates(subset = ['comment_body'])
df_parquet_results_trump.shape
df_parquet_results_trump.head(5)

,comment_body,source,user_location,candidatepoll
0,Why does trump lie?,nyt_comment,Los Angeles,Trump
1,American colonists fought a bloody revolution ...,nyt_comment,Camas WA,Trump
2,"""Oh Bret. You just cant let go can you? Always...",nyt_comment,LINY,Trump
3,"""Trumpism has demonstrated that a large part o...",nyt_comment,usa,Trump
4,"""The Trump administration has been a disaster ...",nyt_comment,California,Trump


In [9]:
path =  "s3://{}/ADS508_project/parquet".format(bucket)

# Call all public discourse data from 
df_parquet_results = wr.s3.read_parquet(
    path, columns=["comment_body", "source", "user_location", "candidatepoll"], partition_filter=b_filter, dataset=True
)

# "user_state",
df_parquet_results_biden = df_parquet_results.drop_duplicates(subset = ['comment_body'])
df_parquet_results_biden.shape
df_parquet_results_biden.head(5)

,comment_body,source,user_location,candidatepoll
0,"""@Dr. Girl We only just barely got BIDEN elect...",nyt_comment,Maryland,Biden
1,I've always thought Joe Biden was underrated. ...,nyt_comment,Indiana,Biden
2,"""I’ve seen a lot of Biden in the last few week...",nyt_comment,Whitestown IN,Biden
3,"""In the midst of today's press conference at t...",nyt_comment,Eureka CA,Biden
4,"""When is Biden going to start campaigning?",nyt_comment,Kentucky,Biden


# Data Exploration before Any Feature Transformation

In [11]:
# Checking for missing values from trump data
df_parquet_results_trump.info()

<class 'pandas.core.frame.DataFrame'>
Index: 670045 entries, 0 to 686590
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   comment_body   670045 non-null  string  
 1   source         670045 non-null  string  
 2   user_location  670045 non-null  string  
 3   candidatepoll  670045 non-null  category
dtypes: category(1), string(3)
memory usage: 21.1 MB


In [12]:
# Check for missing values from  biden data
df_parquet_results_biden.info()

<class 'pandas.core.frame.DataFrame'>
Index: 260435 entries, 0 to 266391
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   comment_body   260435 non-null  string  
 1   source         260435 non-null  string  
 2   user_location  260435 non-null  string  
 3   candidatepoll  260435 non-null  category
dtypes: category(1), string(3)
memory usage: 8.2 MB


In [ ]:
# Data Distributions for Trump and Biden 
df_parquet_results_trump['source'].value_counts()


In [ ]:
# Data Distributions for Trump and Biden 
df_parquet_results_biden['source'].value_counts()

In [ ]:
# Filtering Data by Location 
df_parquet_results_trump['user_location'].value_counts()

In [ ]:
# Top 30 locations of social media activity
df_parquet_results_trump['user_location'].value_counts().head(30)

In [ ]:
df_parquet_results_biden['user_location'].value_counts()

In [ ]:
# Top 30 locations of social media activity
df_parquet_results_biden['user_location'].value_counts().head(30)

# Create EDA Overview Data Visualization

In [ ]:
# Data Sources for Each Candidate
# Code from ChatGPT

import matplotlib.pyplot as plt

candidate = ['Trump','Biden']
twitter_n = [df_parquet_results_trump['source'].value_counts()[0],df_parquet_results_biden['source'].value_counts()[0]]
nyt_n = [df_parquet_results_trump['source'].value_counts()[1],df_parquet_results_biden['source'].value_counts()[1]]

x = range(len(candidate))

# Plotting the bars
plt.bar(x, twitter_n, width=0.4, label='Twitter Mentions', align='center')
plt.bar(x, nyt_n, width=0.4, label='NYT Mentions', align='edge')

# Adding labels
plt.xlabel('Candidates')
plt.ylabel('Number of Mentions')
plt.title('Twitter and NYT Mentions by Candidate')
plt.xticks(x, candidate)
plt.legend()

# Show plot
plt.savefig('DataDist.png')

## Standardizing Text Entries and Location Data

In [10]:
!pip install emoji

  Using cached emoji-2.11.0-py2.py3-none-any.whl.metadata (5.3 kB)
Using cached emoji-2.11.0-py2.py3-none-any.whl (433 kB)


In [11]:
import re
import emoji

# Function to preprocess text data
def preprocess_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    # Replace emojis with text labels
    text = emoji.demojize(text)
    # Remove URLs
    text = re.sub(r'http\S+', ' ', text)
    # Remove "#" from hashtags
    text = re.sub(r'#', '', text)
    # Remove "@" from mentions
    text = re.sub(r'@', '', text)
    # Remove extra spaces while preserving at least one space between words
    text = re.sub(r'\s+', ' ', text)
    # Check if the text consists only of numbers
    if text.isdigit():
        text = "NA"
    return text

#df_parquet_results_biden['user_location'] = df_parquet_results_biden['user_location'].astype(str)
#df_parquet_results_trump['user_location'] = df_parquet_results_trump['user_location'].astype(str)


In [12]:
# Apply preprocessing to text columns in the dataframe
df_parquet_results_biden['clean_text'] = df_parquet_results_biden['comment_body'].apply(preprocess_text)

/tmp/ipykernel_30253/3431105715.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parquet_results_biden['clean_text'] = df_parquet_results_biden['comment_body'].apply(preprocess_text)


In [13]:
df_parquet_results_biden['clean_location'] = df_parquet_results_biden['user_location'].apply(preprocess_text)

/tmp/ipykernel_30253/400958404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parquet_results_biden['clean_location'] = df_parquet_results_biden['user_location'].apply(preprocess_text)


In [14]:
df_parquet_results_trump['clean_text'] = df_parquet_results_trump['comment_body'].apply(preprocess_text)

In [15]:
df_parquet_results_trump['clean_location'] = df_parquet_results_trump['user_location'].apply(preprocess_text)

,comment_body,source,user_location,candidatepoll,clean_text,clean_location
0,Why does trump lie?,nyt_comment,Los Angeles,Trump,Why does trump lie,Los Angeles
1,American colonists fought a bloody revolution ...,nyt_comment,Camas WA,Trump,American colonists fought a bloody revolution ...,Camas WA
2,"""Oh Bret. You just cant let go can you? Always...",nyt_comment,LINY,Trump,Oh Bret You just cant let go can you Always l...,LINY
3,"""Trumpism has demonstrated that a large part o...",nyt_comment,usa,Trump,Trumpism has demonstrated that a large part o...,usa
4,"""The Trump administration has been a disaster ...",nyt_comment,California,Trump,The Trump administration has been a disaster ...,California
5,"""Yep! Trump's ignored most of his administrati...",nyt_comment,minnesota,Trump,Yep Trump s ignored most of his administratio...,minnesota
6,Who is this Trump who can eat people alive in ...,nyt_comment,Florida,Trump,Who is this Trump who can eat people alive in ...,Florida
7,You don’t need to love joe. You just need to d...,nyt_comment,Sunny Cal,Trump,You don t need to love joe You just need to do...,Sunny Cal
8,"""Instead of Trump's worthless nightly news and...",nyt_comment,minnesota,Trump,Instead of Trump s worthless nightly news and...,minnesota
9,"""@John It is so bizarre that a lot of the same...",nyt_comment,Atlanta GA,Trump,John It is so bizarre that a lot of the same ...,Atlanta GA


# Word Clouds with Comment Body Text

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re

# Sample a subset of comments for generating the word cloud
sampled_trump_comments = df_parquet_results_trump['clean_text'].dropna().sample(n=10000, random_state=42)

# Concatenate sampled comments for Trump
trump_comments_text = ' '.join(sampled_trump_comments)

# Define words and patterns to exclude
words_to_exclude = ['trump', 'donald', 'donaldtrump', 'https', 'co', 'st'] 
patterns_to_exclude = [r'\btrump\b', r'\bdonald\b', r'\bdonaldtrump\b', r'https?://\S+']

# Combine words and patterns to exclude
exclude_patterns = '|'.join(words_to_exclude + patterns_to_exclude)

# Preprocess text to remove specific words and patterns
trump_comments_text_cleaned = re.sub(exclude_patterns, '', trump_comments_text, flags=re.IGNORECASE)

# Generate word cloud for Trump comments with a limit on the number of words
wordcloud_trump = WordCloud(width=800, height=400, background_color='white', max_words=100).generate(trump_comments_text_cleaned)

# Plot the word cloud for Trump
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud_trump, interpolation='bilinear')
plt.title('Word Cloud for Trump Comments (Excluding "Trump" and "Donald")')
plt.axis('off')
plt.show()

In [ ]:
# Sample a subset of comments for Biden
sampled_biden_comments = df_parquet_results_biden['clean_text'].dropna().sample(n=40000, random_state=42)

# Concatenate sampled comments for Biden
biden_comments_text = ' '.join(sampled_biden_comments)

# Define words and patterns to exclude
words_to_exclude = ['biden', 'joe', 'joebiden', 'president', 'election', 'https', 'co','amp']  # Exclude common words and URLs
patterns_to_exclude = [r'\bbiden\b', r'\bjoe\b', r'\bjoebiden\b', r'https?://\S+']

# Combine words and patterns to exclude
exclude_patterns = '|'.join(words_to_exclude + patterns_to_exclude)

# Preprocess text to remove specific words and patterns
biden_comments_text_cleaned = re.sub(exclude_patterns, '', biden_comments_text, flags=re.IGNORECASE)

# Generate word cloud for Biden comments with a limit on the number of words
wordcloud_biden = WordCloud(width=800, height=400, background_color='white', max_words=100).generate(biden_comments_text_cleaned)

# Plot the word cloud for Biden
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud_biden, interpolation='bilinear')
plt.title('Word Cloud for Biden Comments (Excluding "Biden" and "Joe")')
plt.axis('off')
plt.show()

# Feature Transformation and Data Cleaning

### Sentiment Labels Using Textblob and Stripping Out Non-English Text Entries with Fasttext.

### This code must be ran on Mac/Linux only. Windows is not compatible. Clean data from this code is stored in an S3 bucket and called below.

In [ ]:
#pip install langdetect

In [ ]:
#pip install fasttext-wheel

In [16]:
#Use pretrained model
#!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin -P /ADS508_GroupProject
#!ls -p /ADS508_GroupProject

  Using cached textblob-0.18.0.post0-py3-none-any.whl.metadata (4.5 kB)
Using cached textblob-0.18.0.post0-py3-none-any.whl (626 kB)


In [ ]:
#import fasttext
# Load the pre-trained language identification model
#model_path = '/ADS508_GroupProject/lid.176.bin'  # Path to the pre-trained language identification model
#model = fasttext.load_model(model_path)

In [ ]:
# Function to detect language using FastText
#def detect_language_fasttext(text):
#    prediction = model.predict(text)
#    language_code = prediction[0][0].split('__label__')[1]
#    return language_code

# Function to analyze tweet sentiment to catch non english words
#def analyze_tweet_sentiment(tweet_text):
#    try:
#        language = detect_language_fasttext(tweet_text)
#        if language == 'en':
#            analysis = TextBlob(tweet_text)
#            return analysis.sentiment.polarity
#        else:
#            return None
#    except:
#        return None



In [17]:
!pip install textblob

from textblob import TextBlob
def analyze_tweet_sentiment(tweet_text):
    analysis = TextBlob(tweet_text)
    # Return sentiment polarity (ranging from -1 to 1)
    return analysis.sentiment.polarity

In [18]:
# Apply sentiment labels to Trump Data
if __name__ == "__main__":
    # Assuming you have a DataFrame named df_tweets with a column 'text' containing tweet text
    df_tweets_trump = df_parquet_results_trump[['clean_text']]
    # Analyze sentiment for each tweet in the DataFrame
    df_tweets_trump['sentiment_score'] = df_tweets_trump['clean_text'].apply(analyze_tweet_sentiment)

# Add sentiment output to parent dataframe
df_parquet_results_trump['sentiment'] = df_tweets_trump['sentiment_score']

/tmp/ipykernel_30253/4017857091.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_trump['sentiment_score'] = df_tweets_trump['clean_text'].apply(analyze_tweet_sentiment)


In [19]:
 # Apply Sentiment labels to Biden Data       
if __name__ == "__main__":
    # Assuming you have a DataFrame named df_tweets with a column 'text' containing tweet text
    df_tweets_biden = df_parquet_results_biden[['clean_text']]
    # Analyze sentiment for each tweet in the DataFrame
    df_tweets_biden['sentiment_score'] = df_tweets_biden['clean_text'].apply(analyze_tweet_sentiment)

# Add sentiment output to parent dataframe
df_parquet_results_biden['sentiment'] = df_tweets_biden['sentiment_score']


/tmp/ipykernel_30253/2765627512.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tweets_biden['sentiment_score'] = df_tweets_biden['clean_text'].apply(analyze_tweet_sentiment)
/tmp/ipykernel_30253/2765627512.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parquet_results_biden['sentiment'] = df_tweets_biden['sentiment_score']


In [23]:
df_parquet_results_trump.head(5)

,comment_body,source,user_location,candidatepoll,clean_text,clean_location,sentiment
0,"""@Dr. Girl We only just barely got BIDEN elect...",nyt_comment,Maryland,Biden,Dr Girl We only just barely got BIDEN elected...,Maryland,0.183333
1,I've always thought Joe Biden was underrated. ...,nyt_comment,Indiana,Biden,I ve always thought Joe Biden was underrated S...,Indiana,0.500000
2,"""I’ve seen a lot of Biden in the last few week...",nyt_comment,Whitestown IN,Biden,I ve seen a lot of Biden in the last few week...,Whitestown IN,0.166667
3,"""In the midst of today's press conference at t...",nyt_comment,Eureka CA,Biden,In the midst of today s press conference at t...,Eureka CA,0.000000
4,"""When is Biden going to start campaigning?",nyt_comment,Kentucky,Biden,When is Biden going to start campaigning,Kentucky,0.000000


## Combine candidate data

In [20]:
df_combined = pd.concat([df_parquet_results_biden, df_parquet_results_trump], ignore_index=True)

### Create sentiment labels

In [21]:
df_combined['sentiment_category'] = df_combined['sentiment'].apply(lambda x: 'negative' if x < 0 else ('positive' if x > 0 else 'neutral'))

In [24]:

df_combined.head()


,comment_body,source,user_location,candidatepoll,clean_text,clean_location,sentiment,sentiment_category
260435,Why does trump lie?,nyt_comment,Los Angeles,Trump,Why does trump lie,Los Angeles,0.000000,neutral
260436,American colonists fought a bloody revolution ...,nyt_comment,Camas WA,Trump,American colonists fought a bloody revolution ...,Camas WA,-0.165909,negative
260437,"""Oh Bret. You just cant let go can you? Always...",nyt_comment,LINY,Trump,Oh Bret You just cant let go can you Always l...,LINY,0.000000,neutral
260438,"""Trumpism has demonstrated that a large part o...",nyt_comment,usa,Trump,Trumpism has demonstrated that a large part o...,usa,0.214286,positive
260439,"""The Trump administration has been a disaster ...",nyt_comment,California,Trump,The Trump administration has been a disaster ...,California,-0.050000,negative


In [40]:
df_combined.shape

(930480, 8)

# Additional EDA with Clean Language Data

## Trump Data

In [ ]:
# Examine Trump Sentiment distribution
df_combined[df_combined['candidatepoll'] == 'Trump']['sentiment'].value_counts()

In [ ]:
df_combined[(df_combined['candidatepoll'] == 'Trump') & (df_combined['sentiment'] < 0)].head(5)

In [ ]:
df_combined[(df_combined['candidatepoll'] == 'Trump') & (df_combined['sentiment'] == 0)].head(5)

In [ ]:
df_combined[(df_combined['candidatepoll'] == 'Trump') & (df_combined['sentiment'] > 0)].head(5)

In [ ]:
plt.hist(df_combined[df_combined['candidatepoll'] == 'Trump']['sentiment'], bins=30, edgecolor='black')

# Adding labels
plt.xlabel('Sentiment Score')
plt.ylabel('Frequency')
plt.title('Distribution of Sentiment Scores for Trump')

# Show plot
plt.savefig('TrumpSentDist.png')

### Biden Data

In [ ]:
df_combined[df_combined['candidatepoll'] == 'Biden']['sentiment'].value_counts()

In [ ]:
df_combined[(df_combined['candidatepoll'] == 'Biden') & (df_combined['sentiment'] < 0)].head(5)

In [ ]:
df_combined[(df_combined['candidatepoll'] == 'Biden') & (df_combined['sentiment'] == 0)].head(5)

In [ ]:
df_combined[(df_combined['candidatepoll'] == 'Biden') & (df_combined['sentiment'] > 0)].head(5)

In [ ]:
plt.hist(df_combined[df_combined['candidatepoll'] == 'Biden']['sentiment'], bins=30, edgecolor='black')

# Adding labels
plt.xlabel('Sentiment Score')
plt.ylabel('Frequency')
plt.title('Distribution of Sentiment Scores for Biden')

# Show plot
plt.savefig('BidenSentDist.png')

## Data Source Proportions with Clean Data

In [ ]:
candidate = ['Trump','Biden']
twitter_n = [df_combined[df_combined['candidatepoll'] == 'Trump']['source'].value_counts()[0], 
             df_combined[df_combined['candidatepoll'] == 'Biden']['source'].value_counts()[0]]
nyt_n = [df_combined[df_combined['candidatepoll'] == 'Trump']['source'].value_counts()[1], 
         df_combined[df_combined['candidatepoll'] == 'Biden']['source'].value_counts()[1]]

x = range(len(candidate))

# Plotting the bars
plt.bar(x, twitter_n, width=0.4, label='Twitter Mentions', align='center')
plt.bar(x, nyt_n, width=0.4, label='NYT Mentions', align='edge')

# Adding labels
plt.xlabel('Candidates')
plt.ylabel('Number of Mentions')
plt.title('Twitter and NYT Mentions by Candidate')
plt.xticks(x, candidate)
plt.legend()

# Show plot
plt.savefig('DataDist.png')

# Additional Feature Engineering

## Create Outcome Variable

In [ ]:
df_combined['SentimentOutcome'] = df_combined['candidatepoll']+"_"+df_combined['sentiment_category']

In [ ]:
import matplotlib.pyplot as plt
category_counts = df_combined['SentimentOutcome'].value_counts()
plt.bar(category_counts.index, category_counts.values)
plt.xlabel('Candidate Sentiment Categories')
plt.xticks(rotation=45)  # Rotate x-axis labels by 45 degrees
plt.ylabel('Count')
plt.title('Frequency of Each Possible Sentiment Outcome')
plt.show()

## Smallest category is Biden_negative. Check number. When using train/test/split with 90/5/5 ratio, we will have to balance categories to observationse each based on 90% of the smallest category.

In [ ]:
 df_combined[df_combined['candidatepoll'] == 'Biden']['SentimentOutcome'].value_counts()

In [ ]:
 df_combined[df_combined['candidatepoll'] == 'Trump']['SentimentOutcome'].value_counts()

## Mapping Location Data and Map EDA. NEED TO TROUBLESHOOT THIS STILL.

In [36]:
df_combined.head(5)


,comment_body,source,user_location,candidatepoll,clean_text,clean_location,sentiment,sentiment_category
0,"""@Dr. Girl We only just barely got BIDEN elect...",nyt_comment,Maryland,Biden,Dr Girl We only just barely got BIDEN elected...,Maryland,0.183333,positive
1,I've always thought Joe Biden was underrated. ...,nyt_comment,Indiana,Biden,I ve always thought Joe Biden was underrated S...,Indiana,0.500000,positive
2,"""I’ve seen a lot of Biden in the last few week...",nyt_comment,Whitestown IN,Biden,I ve seen a lot of Biden in the last few week...,Whitestown IN,0.166667,positive
3,"""In the midst of today's press conference at t...",nyt_comment,Eureka CA,Biden,In the midst of today s press conference at t...,Eureka CA,0.000000,neutral
4,"""When is Biden going to start campaigning?",nyt_comment,Kentucky,Biden,When is Biden going to start campaigning,Kentucky,0.000000,neutral


In [27]:
# Specify the S3 bucket name
prefix = "ADS508_project/cleandata/"

# List of corresponding file names
file_name = "cleandata.csv"

# Create an S3 client
s3 = boto3.client('s3')

#refine df_combined
df_combined_ref = df_combined[['source', 'clean_text', 'clean_location', 'candidatepoll', 'sentiment_category']]
df_combined_ref.head()

,comment_body,source,user_location,candidatepoll,clean_text,clean_location,sentiment,sentiment_category
0,"""@Dr. Girl We only just barely got BIDEN elect...",nyt_comment,Maryland,Biden,Dr Girl We only just barely got BIDEN elected...,Maryland,0.183333,positive
1,I've always thought Joe Biden was underrated. ...,nyt_comment,Indiana,Biden,I ve always thought Joe Biden was underrated S...,Indiana,0.500000,positive
2,"""I’ve seen a lot of Biden in the last few week...",nyt_comment,Whitestown IN,Biden,I ve seen a lot of Biden in the last few week...,Whitestown IN,0.166667,positive
3,"""In the midst of today's press conference at t...",nyt_comment,Eureka CA,Biden,In the midst of today s press conference at t...,Eureka CA,0.000000,neutral
4,"""When is Biden going to start campaigning?",nyt_comment,Kentucky,Biden,When is Biden going to start campaigning,Kentucky,0.000000,neutral


In [28]:
# Upload each file to S3
df_combined_ref.to_csv(file_name, index=False)
s3.upload_file(file_name, bucket, prefix + file_name)

In [29]:
# Create temp table of df_combined
temp_table = "temp_table"
s3_private_path_cleandata = "s3://{}/ADS508_project/cleandata".format(bucket)

statement = """
CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
    source VARCHAR(20),
    clean_text VARCHAR(250),
    clean_location VARCHAR(25),
    candidatepoll VARCHAR(10),
    sentiment_category VARCHAR(10)
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')
""".format(
    database_name, temp_table, s3_private_path_cleandata
)

pd.read_sql(statement, conn)

/tmp/ipykernel_30253/161737226.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [32]:
# map df_combined/ cleandata with uscities by city
temp_table = "temp_table_city"
import time
from datetime import datetime
from time import strftime
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"start: {timestamp}\n")

statement = """
CREATE TABLE IF NOT EXISTS {}.{} AS (
SELECT DISTINCT 
    t.source, 
    t.clean_text, 
    t.clean_location, 
    t.candidatepoll, 
    t.sentiment_category, 
    u.state_id
    FROM  dbpoliticpulsecomment.temp_table t
    LEFT JOIN dbpoliticpulsecomment.uscities u
        ON lower(t.clean_location) = lower(u.city)
    where state_id <> ''
)
""".format(database_name, temp_table)
pd.read_sql(statement, conn)

timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"end: {timestamp}\n")

start: 2024-03-27T17:31:48Z



/tmp/ipykernel_30253/436070304.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


end: 2024-03-27T17:31:53Z



In [33]:
# map df_combined/ cleandata with uscities by statename
temp_table = "temp_table_statename"
import time
from datetime import datetime
from time import strftime
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"start: {timestamp}\n")

statement = """
CREATE TABLE IF NOT EXISTS {}.{} AS (
SELECT DISTINCT 
    t.source, 
    t.clean_text, 
    t.clean_location, 
    t.candidatepoll, 
    t.sentiment_category, 
    u.state_id
    FROM  dbpoliticpulsecomment.temp_table t
    LEFT JOIN dbpoliticpulsecomment.uscities u
        ON lower(t.clean_location) = lower(u.state_name)
    where state_id <> ''
)
""".format(database_name, temp_table)
pd.read_sql(statement, conn)

timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"end: {timestamp}\n")

start: 2024-03-27T17:32:01Z



/tmp/ipykernel_30253/3044688570.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


end: 2024-03-27T17:32:15Z



In [35]:
# map df_combined/ cleandata with uscities by stateid
temp_table = "temp_table_stateid"
import time
from datetime import datetime
from time import strftime
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"start: {timestamp}\n")

statement = """
CREATE TABLE IF NOT EXISTS {}.{} AS (
SELECT DISTINCT 
    t.source, 
    t.clean_text, 
    t.clean_location, 
    t.candidatepoll, 
    t.sentiment_category, 
    u.state_id
    FROM  dbpoliticpulsecomment.temp_table t
    LEFT JOIN dbpoliticpulsecomment.uscities u
        ON lower(t.clean_location) = lower(u.state_id)
    where state_id <> ''
)
""".format(database_name, temp_table)
pd.read_sql(statement, conn)

timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"end: {timestamp}\n")

start: 2024-03-27T17:32:38Z



/tmp/ipykernel_30253/97610970.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


end: 2024-03-27T17:32:47Z



In [36]:
# Create a combine of temp_table_state_id, _statename, _city
table_name = "cleandata"
import time
from datetime import datetime
from time import strftime
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"start: {timestamp}\n")

statement = """
CREATE TABLE IF NOT EXISTS {}.{} AS (
SELECT DISTINCT 
    source, 
    clean_text, 
    clean_location, 
    candidatepoll, 
    sentiment_category, 
    state_id
    FROM  dbpoliticpulsecomment.temp_table_stateid
UNION
SELECT DISTINCT 
    source, 
    clean_text, 
    clean_location, 
    candidatepoll, 
    sentiment_category, 
    state_id
    FROM  dbpoliticpulsecomment.temp_table_city
UNION
SELECT DISTINCT 
    source, 
    clean_text, 
    clean_location, 
    candidatepoll, 
    sentiment_category, 
    state_id
    FROM  dbpoliticpulsecomment.temp_table_statename
)
""".format(database_name, table_name)
pd.read_sql(statement, conn)
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"end: {timestamp}\n")

start: 2024-03-27T17:32:50Z



/tmp/ipykernel_30253/4169751111.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


end: 2024-03-27T17:32:58Z



In [37]:
# Drop all unused tables from Athena
drop_table_names = ["temp_table", "temp_table_statename", "temp_table_stateid", "temp_table_city"]

for table_name in drop_table_names:
    table_name = table_name

    statement = """
    DROP TABLE {}.{}
    """.format(database_name, table_name)
    
    pd.read_sql(statement, conn)

/tmp/ipykernel_30253/2936110994.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)
/tmp/ipykernel_30253/2936110994.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)
/tmp/ipykernel_30253/2936110994.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)
/tmp/ipykernel_30253/2936110994.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objec

In [42]:
#==> SHOULD CREATE ANOTHER PARQUET OR READING FROM THIS? 
table_name = "cleandata"
statement = """SELECT * FROM {}.{}""".format(
    database_name, table_name
)
df_combined_clean = pd.read_sql(statement, conn)
df_combined_clean.head(5)

/tmp/ipykernel_30253/365017009.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_combined_clean = pd.read_sql(statement, conn)


,source,clean_text,clean_location,candidatepoll,sentiment_category,state_id
0,nyt_comment,Compromising Biden is exactly what Trump and ...,NC,Biden,positive,NC
1,nyt_comment,Elizabeth Warren finally showed up I hope it ...,VA,Biden,negative,VA
2,nyt_comment,I m not sure the debates are a good idea for ...,CT,Biden,negative,CT
3,tweet,Is this whole thing sponsored by jeep Presiden...,CO,Biden,positive,CO
4,nyt_comment,vwcdolphins YES A full FBI investigation is i...,FL,Biden,positive,FL


In [43]:
df_combined_clean.shape

(330369, 6)

---> USING PYTHON AND TOOK FOREVER

In [28]:
# Read csv US City from S3 ---> PLANNING TO MOVE IT IN .py AND USE PREPROCESSING JOB FROM SAGEMAKER
git_city_loc = "https://raw.githubusercontent.com/HNStaggs/ADS508_GroupProject/main/uscities.csv"
df_city = pd.read_csv(git_city_loc)

#df_parquet_results_trump['clean_location'].head(15000)
df_parquet_results_trump1 = df_parquet_results_trump.head(100)
df_parquet_results_trump1.head(10)

,comment_body,source,user_location,candidatepoll,clean_text,clean_location,sentiment
0,"""@Dr. Girl We only just barely got BIDEN elect...",nyt_comment,Maryland,Biden,Dr Girl We only just barely got BIDEN elected...,Maryland,0.183333
1,I've always thought Joe Biden was underrated. ...,nyt_comment,Indiana,Biden,I ve always thought Joe Biden was underrated S...,Indiana,0.500000
2,"""I’ve seen a lot of Biden in the last few week...",nyt_comment,Whitestown IN,Biden,I ve seen a lot of Biden in the last few week...,Whitestown IN,0.166667
3,"""In the midst of today's press conference at t...",nyt_comment,Eureka CA,Biden,In the midst of today s press conference at t...,Eureka CA,0.000000
4,"""When is Biden going to start campaigning?",nyt_comment,Kentucky,Biden,When is Biden going to start campaigning,Kentucky,0.000000
5,@Yeah The corroborating evidence is much stron...,nyt_comment,Somewhere,Biden,Yeah The corroborating evidence is much stron...,Somewhere,0.200000
6,What happened to Bloomberg's promise to put hi...,nyt_comment,Los Angeles CA,Biden,What happened to Bloomberg s promise to put hi...,Los Angeles CA,0.250000
7,"""@sharpshin How naive are you? Once Biden won ...",nyt_comment,Rocky Mountains,Biden,sharpshin How naive are you Once Biden won S C,Rocky Mountains,-0.300000
8,"""Maybe the problem in this conversation is bei...",nyt_comment,Portland Oregon,Biden,Maybe the problem in this conversation is bei...,Portland Oregon,-0.126488
9,Biden's on a fast track. A gift - finally some...,nyt_comment,New York,Biden,Biden s on a fast track A gift finally somethi...,New York,0.100000


In [29]:
# PREFER TO RUN IT IN .py with SageMaker processing job
# TRY IT WITH 10000 RECORD took FOREVER

import time
from datetime import datetime
from time import strftime
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
print(f"start: {timestamp}\n")

# Create master location table
git_city_loc = "https://raw.githubusercontent.com/HNStaggs/ADS508_GroupProject/main/uscities.csv"
df_city = pd.read_csv(git_city_loc)
master_location = df_city

# Function to map city and state name to state ID
def map_state_id(city):
    for index, loc in master_location.iterrows():
        if city.lower() == loc['city'].lower() or city.lower() == loc['state_name'].lower():
            return loc['state_id']
    return None

# Apply mapping function to user location table
#df_parquet_results_trump1['final_location'] = df_parquet_results_trump1['clean_location'].apply(map_state_id)  # Use clean location data from primary cleaning step

# Display the user location table with State ID column added
timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")

print(f"end: {timestamp}\n")


start: 2024-03-26T14:49:24Z

end: 2024-03-26T17:48:22Z



/tmp/ipykernel_98/218112506.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parquet_results_trump1['final_location'] = df_parquet_results_trump1['clean_location'].apply(map_state_id)  # Use clean location data from primary cleaning step


In [31]:
df_parquet_results_trump1.shape

(10000, 8)

In [ ]:
### working progress.. still need to check##
from sagemaker.processing import ScriptProcessor

# Create a ScriptProcessor object
script_processor = ScriptProcessor(base_job_name='data-processing',
                                   image_uri=None,
                                   command=['python3'],
                                   instance_count=1,
                                   instance_type='ml.m5.xlarge',
                                   role=role)

# Specify the entry script
entry_script = 's3://sagemaker-us-west-2-471112815505/ADS508_project/script/process_loc.py'

# Run the processing job
script_processor.run(code=entry_script)


In [ ]:
# Data Vis
# # creat colored maps by state based on sentiment

In [ ]:
# Determime sample size from each state

In [ ]:
# Determine portion of Trump/Biden for each state

# Data Partition

### Assign input and outcome variables

In [ ]:
X = df_combined[['clean_text']]  # Add location back in later
y = df_combined['SentimentOutcome']

### Create a 90/5/5 data split

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into 90% train and 10% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

# Split remaining data (90% train) into 90% train and 10% validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=1/9, random_state=1)

### Further undersample the training set so that each level of the outcome variable is equal

In [ ]:
#!pip install imblearn
from imblearn.under_sampling import RandomUnderSampler

# Undersample the training set to balance the six categorical outcome levels
undersampler = RandomUnderSampler(sampling_strategy='all', random_state=1)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

## Verify split datasets size

In [ ]:
X_train_resampled.info()

In [ ]:
X_test.info()

In [ ]:
X_val.info()

In [ ]:
X_train_resampled.shape, X_test.shape, X_val.shape

In [ ]:
X_train_resampled.head()

In [ ]:
X_test.head()

In [ ]:
X_val.head()

## Verify balance of training set

In [ ]:
y_train_resampled.value_counts()

## Average length of comments. Limit word vectors to median length of 15.

In [ ]:
# Function to count string word length
def count_word_length(text):
    words = text.split()
    return len(words)

# Add column to DataFrame
df_combined['word_length'] = df_combined['clean_text'].apply(lambda x: count_word_length(x))

In [ ]:
import numpy as np
median_word_length = np.median(df_combined['word_length'])

# Plot histogram of word lengths
plt.hist(df_combined['word_length'], bins=range(min(df_combined['word_length']), max(df_combined['word_length']) + 1), edgecolor='black')
plt.axvline(median_word_length, color='red', linestyle='dashed', linewidth=1)  # Add vertical line at median
plt.text(median_word_length, plt.ylim()[1] * 0.9, f'Median: {median_word_length:.2f}', color='red')  # Add label for median
plt.xlabel('Word Length')
plt.ylabel('Frequency')
plt.title('Histogram of Word Lengths')
plt.grid(True)
plt.show()

# Text Entry Preprocessing and Data Transformation Pipeline: Will Need to Add Categorical One-Hot Encoding if we use Location Data

In [ ]:
#!pip install gensim
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from gensim.models import Word2Vec

# Define custom transformer for Word2Vec embeddings
class Word2VecTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, window=5, min_count=1, workers=4):
        self.word2vec_model = Word2Vec(window=window, min_count=min_count, workers=workers)
    
    def fit(self, X, y=None):
        # Train Word2Vec model
        X_word_lists = [text.split()[:15] for text in X]  # Limit to first 15 words
        self.word2vec_model.build_vocab(X_word_lists)
        self.word2vec_model.train(X_word_lists, total_examples=len(X_word_lists), epochs=10)
        return self
    
    def transform(self, X):
        # Limit text to first 15 words and split into word lists
        X_word_lists = [text.split()[:15] for text in X]
        # Initialize a list to store word embeddings
        embeddings = []
        # Iterate over each word list
        for word_list in X_word_lists:
            # Initialize a list to store word embeddings for this word list
            word_embeddings = []
            # Iterate over each word in the word list
            for word in word_list:
                # Check if the word exists in the Word2Vec model's vocabulary
                if word in self.word2vec_model.wv:
                    # Retrieve the word embedding and append it to the list
                    word_embeddings.append(self.word2vec_model.wv[word])
                else:
                    # If the word doesn't exist in the vocabulary, append zeros
                    word_embeddings.append([0] * self.word2vec_model.vector_size)
            # Append the word embeddings for this word list to the embeddings list
            embeddings.append(word_embeddings)
        return embeddings

# Define preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('word2vec', Word2VecTransformer())
])

In [ ]:
# Fit and transform on the training set
X_train_transformed = pd.DataFrame(preprocessing_pipeline.fit_transform(X_train_resampled['clean_text']))

In [ ]:
X_test_transformed = pd.DataFrame(preprocessing_pipeline.transform(X_test['clean_text'])) 

In [ ]:
X_val_transformed = pd.DataFrame(preprocessing_pipeline.transform(X_val['clean_text']))

In [ ]:
X_train_transformed.head()

In [ ]:
X_test_transformed.head()

In [ ]:
X_val_transformed.head()

# Store Data For Modeling in Bucket

In [ ]:
#import boto3

# Specify the S3 bucket name
bucket_name = '508group_modeling'
    
# List of DataFrame objects
dataframes = [X_train_transformed, y_train, X_test_transformed, y_test, X_val_transformed, y_val]

# List of corresponding file names
file_names = ['X_train_transformed.csv', 'y_train.csv', 'X_test_transformed.csv', 'y_test.csv', 'X_val_transformed.csv', 'y_val.csv']

# Create an S3 client
s3 = boto3.client('s3')

# Upload each file to S3
for df, file_name in zip(dataframes, file_names):
    # Write DataFrame to a CSV file
    df.to_csv(file_name, index=False)
    
    # Upload CSV file to S3
    with open(file_name, "rb") as f:
        s3.upload_fileobj(f, bucket_name, file_name)

    print(f"DataFrame saved as {file_name} in S3 bucket: {bucket_name}")

# Import Data for Modeling

In [ ]:
import boto3
import pandas as pd

# Specify the S3 bucket name
bucket_name = '508group_modeling'

# List of file names in S3
file_names = ['X_train_transformed.csv', 'y_train.csv', 'X_test_transformed.csv', 
              'y_test.csv', 'X_val_transformed.csv', 'y_val.csv']

# Create an S3 client
s3 = boto3.client('s3')

# Initialize variables to store DataFrames
X_train = None
y_train = None
X_test = None
y_test = None
X_val = None
y_val = None

# Iterate over each file name
for i, file_name in enumerate(file_names):
    # Generate the URL for the file in S3
    file_url = f's3://{bucket_name}/{file_name}'
    
    # Read the CSV file directly into a DataFrame
    df = pd.read_csv(file_url)
    
    # Assign the DataFrame to the corresponding variable based on index
    if i == 0:
        X_train = df
    elif i == 1:
        y_train = df
    elif i == 2:
        X_test = df
    elif i == 3:
        y_test = df
    elif i == 4:
        X_val = df
    elif i == 5:
        y_val = df

## Verify contents of DFs

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_test.head()

In [ ]:
y_test.head()

In [ ]:
X_val.head()

In [ ]:
y_val.head()

## Run Data Bias Analysis

In [ ]:
# Create folder for specific timestamp
import time

timestamp = int(time.time())

bias_data_s3_uri = sess.upload_data(bucket=bucket, key_prefix="ADS508_project/bias-detection-{}".format(timestamp), path=path)
bias_data_s3_uri

In [ ]:
!aws s3 ls $bias_data_s3_uri

In [ ]:
balanced_bias_data_s3_uri = sess.upload_data(
    bucket=bucket, key_prefix="ADS508_project/bias-detection-{}".format(timestamp), path=path_balance
)
balanced_bias_data_s3_uri

In [ ]:
!aws s3 ls $balanced_bias_data_s3_uri

In [ ]:
balanced_bias_data_jsonlines_s3_uri = sess.upload_data(
    bucket=bucket, key_prefix="ADS508_project/bias-detection-{}".format(timestamp), path=path_jsonlines
)
balanced_bias_data_jsonlines_s3_uri

In [ ]:
!aws s3 ls $balanced_bias_data_jsonlines_s3_uri

In [ ]:
#!pip install --force-reinstall -q smclarify

from smclarify.bias import report
from typing import Dict
from collections import defaultdict
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!aws s3 cp $bias_data_s3_uri ./data-clarify/

In [ ]:
!aws s3 cp $balanced_bias_data_s3_uri ./data-clarify/

### Calculate Bias Metrics for all Data
#### might not needed from ===>

facet_column = report.FacetColumn(name="candidatepoll")

label_column = report.LabelColumn(
    name="sentiment_category", 
    series=df_combined["sentiment_category"],
    positive_label_values=[5]
)

In [ ]:
report.bias_report(
    df=df_combined, 
    facet_column=facet_column, 
    label_column=label_column, 
    stage_type=report.StageType.PRE_TRAINING, 
    metrics=["CI", "DPL", "KL", "JS", "LP", "TVD", "KS"]
)

### Calculate Bias Metrics for Balanced Data

In [ ]:
from smclarify.bias import report

facet_column = report.FacetColumn(name="candidatepoll")

label_column = report.LabelColumn(
    name="sentiment_category", 
    series=df_balanced["sentiment_category"], 
    positive_label_values=[5]
)

In [ ]:
report.bias_report(
    df=df_balanced, 
    facet_column=facet_column, 
    label_column=label_column, 
    stage_type=report.StageType.PRE_TRAINING, 
    metrics=["CI", "DPL", "KL", "JS", "LP", "TVD", "KS"]
)

#### <=== TO MIGHT NOT NEEDED

### Run Data Bias Analysis (Pre-training)

In [ ]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, 
    instance_count=1, 
    instance_type="ml.c5.xlarge", 
    sagemaker_session=sess
)

#### Pre-Training Bias

In [ ]:
bias_report_output_path = "s3://{}/ADS508_project/clarify".format(bucket)

bias_data_config = clarify.DataConfig(
    s3_data_input_path=bias_data_s3_uri,
    s3_output_path=bias_report_output_path,
    label="sentiment_category",
    headers=df_combined.columns.to_list(),
    dataset_type="text/csv",
)


In [ ]:
# Trump
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[5],
    facet_name="candidatepoll",
    facet_values_or_threshold=["Trump"],
)

clarify_processor.run_pre_training_bias(
    data_config=bias_data_config, 
    data_bias_config=bias_config, 
    methods=["CI", "DPL", "KL", "JS", "LP", "TVD", "KS"],
    wait=False, 
    logs=False
)

run_pre_training_bias_processing_job_name = clarify_processor.latest_job.job_name
run_pre_training_bias_processing_job_name



In [ ]:
# Biden
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[5],
    facet_name="candidatepoll",
    facet_values_or_threshold=["Biden"],
)

clarify_processor.run_pre_training_bias(
    data_config=bias_data_config, 
    data_bias_config=bias_config, 
    methods=["CI", "DPL", "KL", "JS", "LP", "TVD", "KS"],
    wait=False, 
    logs=False
)

run_pre_training_bias_processing_job_name = clarify_processor.latest_job.job_name
run_pre_training_bias_processing_job_name


#### Download report

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, run_pre_training_bias_processing_job_name
        )
    )
)
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, run_pre_training_bias_processing_job_name
        )
    )
)

from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, run_pre_training_bias_processing_job_name, region
        )
    )
)

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=run_pre_training_bias_processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [ ]:
!aws s3 ls $bias_report_output_path/

In [ ]:
!aws s3 cp --recursive $bias_report_output_path ./generated_bias_report/

In [ ]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="./generated_bias_report/report.html">Bias Report</a></b>'))

In [ ]:
path = "./data-clarify/amazon_reviews_us_giftcards_software_videogames.csv"
df.to_csv(path, index=False, header=True)

In [ ]:
_testing